### 1. Install TF Lite model maker

Note: To install 'TF Lite model maker' you must have installed MSVC C++ build Tools.

Link: https://visualstudio.microsoft.com/visual-cpp-build-tools/

In [ ]:
if 1 == 1:
    %pip install -q tflite-model-maker
    %pip install -q pycocotools

### 2. Import the required packages

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### 3. Gathering images and create dataset

1. create a folder for raw images of object.
2. use labelImg tool for marking object in images.
3. convert to csv file with following tensorflow format:
    <TRAIN/VALIDATION/TEST>,<FILE_PATH>,<annotation>,<XMIN>,<YMIN>,,,<XMAX>,<YMAX>,,

### 4. Train object detection Model

In [ ]:
spec = model_spec.get('efficientdet_lite0')
train_data, validation_data, test_data = object_detector.DataLoader.from_csv('dataset/dataset.csv')

In [ ]:
model = object_detector.create(train_data, model_spec=spec, epochs=10, batch_size=8, train_whole_model=True, validation_data=validation_data)

In [ ]:
model.evaluate(validation_data)

### 5. Export Trained Model File

In [ ]:
model.export(export_dir='.')
model.evaluate_tflite('model.tflite', test_data)

### 6. Save Labels

In [ ]:
# Load the labels into a files
label_map = model.model_spec.config.label_map
l_file = open('labels.txt', 'w')
for label_id, label_name in label_map.as_dict().items():
  l_file.write(label_name + '\n')
l_file.close()